<a href="https://colab.research.google.com/github/xiaoyufan/speech-data-augmentation/blob/main/train_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/train-baseline.ipynb)

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Configurations

In [2]:
DRIVE_PROJECT_ROOT_PATH = '/content/drive/MyDrive/nlp-project'
DATASET_PATH = f'{DRIVE_PROJECT_ROOT_PATH}/tensorflow-speech-recognition-challenge'
DEEPSPEECH_LOG_LEVEL = '0'
DEEPSPEECH_PATH = '/content/DeepSpeech'
LOGGING_LEVEL = 'DEBUG'
USE_GPU = True

In [3]:
TRAIN_FILES_PATH = f'{DATASET_PATH}/playground/training_files.csv'
TEST_FILES_PATH = f'{DATASET_PATH}/playground/testing_files.csv'

# For now, we don't use test/audio because its audio files don't have transcriptions.
# Instead, we split train/audio into training set and testing set.
WAV_TRAIN_DIR = f'{DATASET_PATH}/playground/audio'

## Create logger

In [4]:
from importlib import reload
import logging
import sys

reload(logging)

logger = logging.getLogger('baseline')
logger.setLevel(LOGGING_LEVEL)

formatter = logging.Formatter('[%(asctime)s - logger %(name)s - %(levelname)s] %(message)s')

ch = logging.StreamHandler(sys.stdout)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.debug('debug test')
logger.info('info test')

[2020-12-09 06:23:45,576 - logger baseline - DEBUG] debug test
[2020-12-09 06:23:45,577 - logger baseline - INFO] info test


## Setup

### Install packages

In [5]:
%%bash -s "$DEEPSPEECH_PATH"
DEEPSPEECH_PATH=$1

if [ ! -d "$DEEPSPEECH_PATH" ] ; then
  git clone --branch v0.9.2 https://github.com/mozilla/DeepSpeech $DEEPSPEECH_PATH
fi

cd $DEEPSPEECH_PATH
pip install --upgrade pip==20.2.2 wheel==0.34.2 setuptools==49.6.0
pip install --upgrade -e .

# pip uninstall tensorflow -y
pip install --upgrade tensorflow==1.15.4
pip install tensorflow-gpu==1.15.4

# Install other packages
pip install pandas
# tensorflow 1.15.4 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
pip install --upgrade numpy==1.16.0

Requirement already up-to-date: pip==20.2.2 in /usr/local/lib/python3.6/dist-packages (20.2.2)
Requirement already up-to-date: wheel==0.34.2 in /usr/local/lib/python3.6/dist-packages (0.34.2)
Requirement already up-to-date: setuptools==49.6.0 in /usr/local/lib/python3.6/dist-packages (49.6.0)
Obtaining file:///content/DeepSpeech
  Attempting uninstall: deepspeech-training
    Found existing installation: deepspeech-training 0.9.2
    Can't uninstall 'deepspeech-training'. No files were found to uninstall.
  Running setup.py develop for deepspeech-training
Requirement already up-to-date: tensorflow==1.15.4 in /usr/local/lib/python3.6/dist-packages (1.15.4)
Requirement already up-to-date: numpy==1.16.0 in /usr/local/lib/python3.6/dist-packages (1.16.0)


### Verify tensorflow can run on GPU

In [6]:
import tensorflow as tf
logger.info(f'tensorflow version: {tf.__version__}')

if tf.test.gpu_device_name(): 
    logger.info('Using Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   logger.info("Not using GPU")

[2020-12-09 06:24:10,011 - logger baseline - INFO] tensorflow version: 1.15.4
[2020-12-09 06:24:10,310 - logger baseline - INFO] Using Default GPU Device: /device:GPU:0


## Transform tensorflow-challenge dataset into DeepSpeech format

In [7]:
# Importer

import os
import sys
import pandas

from pathlib import Path


def load_testing_files():
    testing_list_path = f'{DATASET_PATH}/train/testing_list.txt'

    with open(testing_list_path) as file:
        testing_files = [line.rstrip() for line in file]
        return testing_files


def generate_files_list(testing_files):
    if (os.path.exists(TRAIN_FILES_PATH) and
            os.path.exists(TEST_FILES_PATH)):
        logger.info(f'Skipping transforming data. Data files {TRAIN_FILES_PATH} and {TEST_FILES_PATH} already exist. ')
        return

    COLUMNS = ['wav_filename', 'wav_filesize', 'transcript']
    training_data = []
    testing_data = []

    for path in Path(WAV_TRAIN_DIR).rglob('*.wav'):
        wav_path_relative_to_wav_dir = str(path.relative_to(WAV_TRAIN_DIR))
        wav_set = None

        if wav_path_relative_to_wav_dir in testing_files:
            wav_set = 'testing'
        else:
            wav_set = 'training'

        wav_filename = path.name
        transcript = wav_path_relative_to_wav_dir.split('/')[0]

        logger.debug(
            f'Wav: {wav_path_relative_to_wav_dir}; Dataset: {wav_set}; Transcript: {transcript}')

        data = (str(path),
                path.stat().st_size,
                transcript)

        if wav_set == 'training':
            if transcript == '_background_noise_':
                logger.debug(f'SKipping adding {wav_filename} to files list.')
                continue

            training_data.append(data)
            
        if wav_set == 'testing':
            testing_data.append(data)

    training_df = pandas.DataFrame(
        data=training_data,
        columns=COLUMNS,
    )
    training_df.to_csv(os.path.join(TRAIN_FILES_PATH), index=False)
    logger.info(f'Train data files generated at {TRAIN_FILES_PATH}.')

    testing_df = pandas.DataFrame(
        data=testing_data,
        columns=COLUMNS,
    )
    testing_df.to_csv(os.path.join(TEST_FILES_PATH), index=False)
    logger.info(f'Test data files generated at {TRAIN_FILES_PATH}.')


def transform_data():
    # Load testing files list
    testing_files = load_testing_files()
    # Generate files list
    generate_files_list(testing_files)


transform_data()

[2020-12-09 06:24:10,387 - logger baseline - INFO] Skipping transforming data. Data files /content/drive/MyDrive/nlp-project/tensorflow-speech-recognition-challenge/playground/training_files.csv and /content/drive/MyDrive/nlp-project/tensorflow-speech-recognition-challenge/playground/testing_files.csv already exist. 


## Train a baseline model

Train a DeepSpeech model with Kaggle Tensorflow challenge's dataset to establish the baseline.

In [8]:
%%bash -s "$DEEPSPEECH_PATH" "$TRAIN_FILES_PATH" "$TEST_FILES_PATH" "$DRIVE_PROJECT_ROOT_PATH" "$DEEPSPEECH_LOG_LEVEL"
DEEPSPEECH_PATH=$1
TRAIN_FILES_PATH=$2
TEST_FILES_PATH=$3
DRIVE_PROJECT_ROOT_PATH=$4
DEEPSPEECH_LOG_LEVEL=$5

cd $DEEPSPEECH_PATH
python DeepSpeech.py \
  --alphabet_config_path=data/alphabet.txt \
  --train_files "$TRAIN_FILES_PATH" \
  --test_files "$TEST_FILES_PATH" \
  --checkpoint_dir "$DRIVE_PROJECT_ROOT_PATH/xiaoyu-baseline/checkpoints" \
  --export_dir "$DRIVE_PROJECT_ROOT_PATH/xiaoyu-baseline/models" \
  --log_level "$DEEPSPEECH_LOG_LEVEL"

D Session opened.
I Could not find best validating checkpoint.
I Loading most recent checkpoint from /content/drive/MyDrive/nlp-project/xiaoyu-baseline/checkpoints/train-53
I Loading variable from checkpoint: beta1_power
I Loading variable from checkpoint: beta2_power
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/bias
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/bias/Adam
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/bias/Adam_1
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel/Adam
I Loading variable from checkpoint: cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel/Adam_1
I Loading variable from checkpoint: global_step
I Loading variable from

2020-12-09 06:24:12.587361: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-12-09 06:24:12.594705: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-12-09 06:24:12.595067: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x235af40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-09 06:24:12.595091: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-12-09 06:24:12.597235: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-09 06:24:12.710393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-09 06:24:12.711095: I ten